## Program to implement Random Forest:

Imports needed: 

In [45]:
from random import seed
from random import randrange
from csv import reader
import pandas as pd
import numpy as np
from math import sqrt

Function to load the csv file: 

In [46]:
def load_csv(filename1):
    file1 = open(filename1, "rt")
    lines1 = reader(file1)
    dataset1 = list(lines1)
    return dataset1

Function to convert the String values to floats:

In [47]:
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

Function to convert String values to integer:

In [48]:
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

Function to divide the dataset into k folds(k=n_folds here):

In [49]:
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

Function to calculate accuracy percentage:

In [50]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

Function to evaluate an algorithm using cross validation split:

In [51]:
def evaluate_algorithm(dataset, algorithm, *args):
#     folds = cross_validation_split(dataset, n_folds)
#     scores = list()
#     for fold in folds:
#         train_set = list(folds)
#         train_set.remove(fold)
#         train_set = sum(train_set, [])
#         test_set = list()
#         for row in fold:
#             row_copy = list(row)
#             test_set.append(row_copy)
#             row_copy[-1] = None
#         predicted = algorithm(train_set, test_set, *args)
#         actual = [row[-1] for row in fold]
#         accuracy = accuracy_metric(actual, predicted)
#         scores.append(accuracy)

    dataset_copy=list(dataset)
    train_set=dataset_copy[:int(len(dataset_copy)*(7/10))] #Training data is first 70%
    test_set=dataset_copy[int(len(dataset_copy)*(7/10)):] #Test data is last 30%
    
    predicted = algorithm(train_set, test_set, *args)
    actual = [row[-1] for row in test_set]
    accuracy = accuracy_metric(actual, predicted)
    return accuracy

Function to split a dataset based on an attribute and an attribute value

In [52]:
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

Function to calculate the Gini index for a split dataset

In [53]:
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

Function to select the best split point for a dataset

In [54]:
def get_split1(dataset, n_features):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	features = list()
	while len(features) < n_features:
		index = randrange(len(dataset[0])-1) #Generating sample of features without replacements
		if index not in features:
			features.append(index)
	for index in features:
		for row in dataset: #Checking best possible split for among all features
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [55]:
def get_split2(dataset):
	class_values = list(set(row[-1] for row in dataset)) #Assumed that row[-1] means the last column value is the class
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	for index in range(len(dataset[0])-1): #Index is pointing to the columns of the dataset
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:  #Get the lowest ging score among all possible splits
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

Function to create a terminal node value

In [56]:
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

Function to create child splits for a node or make terminal

In [57]:
def split1(node, max_depth, min_size, n_features, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split1(left, n_features)
		split1(node['left'], max_depth, min_size, n_features, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split1(right, n_features)
		split1(node['right'], max_depth, min_size, n_features, depth+1)

In [58]:
def split2(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split2(left)
		split2(node['left'], max_depth, min_size, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split2(right)
		split2(node['right'], max_depth, min_size, depth+1)

Function to build a decision tree(Random Forest version)

In [59]:
def build_tree1(train, max_depth, min_size, n_features):  #Here train is actually the random subsample
	root = get_split1(train, n_features)
	split1(root, max_depth, min_size, n_features, 1) #we are assigning the depth as 1 for the first time
	return root


Function to build a decision tree(Normal Decision Tree version)

In [60]:
def build_tree2(train, max_depth, min_size):
	root = get_split2(train)
	split2(root, max_depth, min_size, 1)
	return root

Function to make a prediction with a decision tree

In [61]:
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

Function to create a random sub-sample:

In [76]:
def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
#     while len(sample) < n_sample:
#         index = randrange(len(dataset)) #Generating sample of rows without replacement
#         sample.append(dataset[index])
        
    index=np.random.choice(len(dataset),n_sample,replace=True) #Generating sample of rows with replacement
    for l1 in index:
        sample.append(dataset[l1])
        
    return sample

Function to make predictions from a list of bagged trees:

In [63]:
def bagging_predict(trees, row):
	predictions = [predict(tree, row) for tree in trees]
	return max(set(predictions), key=predictions.count) #So we will return the prediction which has highest vote combinedly

Function for the random forest algorithm:

In [64]:
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
	trees = list()
	for i in range(n_trees): #so i defines number of trees that will be made
		sample = subsample(train, sample_size)
		tree = build_tree1(sample, max_depth, min_size, n_features)
		trees.append(tree)
	predictions = [bagging_predict(trees, row) for row in test]
	return(predictions)

Function for Decision Tree:

In [65]:
def decision_tree(train, test, max_depth, min_size):
	tree = build_tree2(train, max_depth, min_size)
	predictions = list()
	for row in test:
		prediction = predict(tree, row)
		predictions.append(prediction)
	return(predictions)

Comparison between Decision tree and Random Forest on IRIS dataset

In [80]:
seed(2)
# load and prepare data
filename = 'Iris_Dataset.csv'
dataset = load_csv(filename)
# convert string attributes and class column to integers
for i in range(0,len(dataset[0])-1):
    str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
# n_folds = 5
max_depth = 10 #Maximum number of nodes allowed from the root in the decision tree towards the leaf
min_size = 5 #Minimum number of rows allowed under a same node
sample_size=1.0 #This is the ratio of samples size to the actual dataset's size(number of rows)
n_features=int(sqrt(len(dataset[0])-1))
n_trees=20

accuracy1 = evaluate_algorithm(dataset, random_forest, max_depth, min_size, sample_size, n_trees, n_features)
accuracy2 = evaluate_algorithm(dataset, decision_tree, max_depth, min_size)

print('Accuracy of the Iris dataset for Decision Tree is: %.3f%%' % (accuracy2))
print('\nNumber of estimators: %d' % n_trees)
# print('Scores: %s' % scores)
print('Accuracy of the Iris dataset for Random Forest is: %.3f%%' % (accuracy1))

Accuracy of the Iris dataset for Decision Tree is: 73.333%

Number of estimators: 20
Accuracy of the Iris dataset for Random Forest is: 77.778%


References:
[1]  https://machinelearningmastery.com/implement-random-forest-scratch-python/ . [Accessed: Jan. 20, 2019].